In [1]:
# import libraries
import pandas as pd
import numpy as np 
import datetime
import pickle
import logging
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost as xgb


In [2]:
warnings.filterwarnings("ignore")

In [3]:
# Initialize the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [4]:
def load_data(path):
    try:
        df = pd.read_csv(path, low_memory=False)
        return df
    except Exception as e:
        logger.info(f"Error on Loding Data {e}")

In [5]:
path = "../data/02_intermediate/train_clean.csv"
df = load_data(path)

In [6]:
df.head()

,Unnamed: 0,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,...,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Day,Week,Month,Year,Season
0,0,1,5,2015-07-31,5263.0,555.0,1,1,0,1,...,2008.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct",31,31,7,2015,Summer
1,1,2,5,2015-07-31,6064.0,625.0,1,1,0,1,...,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct",31,31,7,2015,Summer
2,2,3,5,2015-07-31,8314.0,821.0,1,1,0,1,...,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct",31,31,7,2015,Summer
3,3,4,5,2015-07-31,13995.0,NaN,1,1,0,1,...,2009.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct",31,31,7,2015,Summer
4,4,5,5,2015-07-31,4822.0,559.0,1,1,0,1,...,2015.0,0,22.0,2012.0,"Jan,Apr,Jul,Oct",31,31,7,2015,Summer


In [7]:
logger.info("Droping unecessary coulmuns")
df = df.drop(['Unnamed: 0', 'Date'], axis=1)

2024-06-02 00:01:29,327 - INFO - Droping unecessary coulmuns


In [8]:
# Preprocessing pipeline
def create_preprocessing_pipeline(numeric_features, categorical_features):
    logger.info("Creating preprocessing pipeline")
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])
    
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)])
    
    return preprocessor

In [ ]:
# Train and evaluate the model
def train_evaluate_model(df, target_column):
    logger.info("Starting model training and evaluation")
    
    # Define features and target
    X = df.drop(columns=[target_column])
    y = df[target_column]
    
    # Define numeric and categorical features
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Create preprocessing pipeline
    preprocessor = create_preprocessing_pipeline(numeric_features, categorical_features)
    
    # Preprocess the features
    logger.info("Preprocessing the features")
    X_preprocessed = preprocessor.fit_transform(X)
    
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X_preprocessed, y, test_size=0.2, random_state=42)
    
    # Create regression matrices
    dtrain_reg = xgb.DMatrix(X_train, y_train)
    dtest_reg = xgb.DMatrix(X_test, y_test)
    
    # Define hyperparameters
    params = {"objective": "reg:squarederror", "tree_method": "hist"}
    n = 5000

    evals = [(dtest_reg, "validation"), (dtrain_reg, "train")]
    model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
    evals=evals,
    verbose_eval=50,
    # Activate early stopping
    early_stopping_rounds=50)
    
    # Train XGBoost model
    logger.info("Training XGBoost model")
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Make predictions
    logger.info("Making predictions")
    y_pred = model.predict(X_test)
    
    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    logger.info(f'Root_Mean Squared Error: {rmse}')
    
    return preprocessor, prophet_model, model